# Spam Classifier
## Assignment Preamble
Please ensure you carefully read all of the details and instructions on the assignment page, this section, and the rest of the notebook. If anything is unclear at any time please post on the forum or ask a tutor well in advance of the assignment deadline.

In addition to all of the instructions in the body of the assignment below, you must also follow the following technical instructions for all assignments in this unit. *Failure to do so may result in a grade of zero.*
* [At the bottom of the page](#Submission-Test) is some code which checks you meet the submission requirements. You **must** ensure that this runs correctly before submission.
* Do not modify or delete any of the cells that are marked as test cells, even if they appear to be empty.
* Do not duplicate any cells in the notebook – this can break the marking script. Instead, insert a new cell (e.g. from the menu) and copy across any contents as necessary.

Remember to save and backup your work regularly, and double-check you are submitting the correct version.

This notebook is the primary reference for your submission. You may write code in separate `.py` files but it must be clearly imported into the notebook so that it runs without needing to reference those files, and you must explain clearly what functionality is contained in those files (through comments, markdown cells, etc).

As always, **the work you submit for this assignment must be entirely your own.** Do not copy or work with other students. Do not copy answers that you find online. These assignments are designed to help improve your understanding first and foremost – the process of doing the assignment is part of *learning*. They are also used to assess your ability, and so you must uphold academic integrity. Submitting plagiarised work risks your entire place on your degree.

**The pass mark for this assignment is 40%.** We expect that students, on average, will be able to produce a submission which gets a mark between 50-70% within the normal workload allocation for the unit, but this will vary depending on individual backgrounds. Please ask for help if you are struggling.

## Getting Started
Spam refers to unwanted email, often in the form of advertisements. In the literature, an email that is **not** spam is called *ham*. Most email providers offer automatic spam filtering, where spam emails will be moved to a separate inbox based on their contents. Of course this requires being able to scan an email and determine whether it is spam or ham, a classification problem. This is the subject of this assignment.

This assignment has two parts. Each part is worth 50% of the overall grade for this assignment.

For part one you will write a supervised learning based classifier to determine whether a given email is spam or ham. You must write and submit the code in this notebook. The training data is provided for you. You may use any classification method. Marks will be awarded primarily based on the accuracy of your classifier on unseen test data, but there are also marks for estimating how accurate you think your classifier will be.

In part two you will produce a short video explaining your implementation, any decisions or extensions you made, and what parameter values you used. This part is explained in more detail on the assignment page. The video file must be submitted with your assignment.

### Choice of Algorithm
While the classification method is a completely free choice, the assignment folder includes [a separate notebook file](data/naivebayes.ipynb) which can help you implement a Naïve Bayes solution. If you do use this notebook, you are still responsible for porting your code into *this* notebook for submission. A good implementation should give a high  enough accuracy to get a good grade on this section (50-70%).

You could also consider a k-nearest neighbour algorithm, but this may be less accurate. Logistic regression is another option that you may wish to consider.

If you are looking to go beyond the scope of the unit, you might be interested in building something more advanced, like an artificial neural network. This is possible just using `numpy`, but will require significant self-directed learning. *Extensions like this are left unguided and are not factored into the unit workload estimates.*

**Note:** you may use helper functions in libraries like `numpy` or `scipy`, but you **must not** import code which builds entire models for you. This includes but is not limited to use of libraries like `scikit-learn`, `tensorflow`, or `pytorch` – there will be plenty of opportunities for these libraries in later units. The point of this assignment is to understand code the actual algorithm yourself. ***If you are in any doubt about any particular library or function please ask a tutor.*** Submissions which ignore this will receive penalties or even zero marks.

If you choose to implement more than one algorithm, please feel free to include your code and talk about it in part two (your video presentation), but only the code in this notebook will be used in the automated testing.

## Training Data
The training data is described below and has 1000 rows. There is also a 500 row set of test data. These are functionally identical to the training data, they are just in a separate csv file to encourage you to split out your training and test data. You should consider how to best make use of all available data without overfitting, and to help produce an unbiased estimate for your classifier's accuracy.

The cell below loads the training data into a variable called `training_spam`.

In [1]:
import numpy as np
from IPython.display import HTML,Javascript, display

training_spam = np.loadtxt(open("data/training_spam.csv"), delimiter=",").astype(int)
print("Shape of the spam training data set:", training_spam.shape)
print(training_spam)

Shape of the spam training data set: (1500, 55)
[[1 0 0 ... 1 1 1]
 [1 1 0 ... 1 1 1]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 1]
 [1 1 1 ... 1 1 0]
 [1 0 0 ... 1 1 1]]


Your training set consists of 1000 rows and 55 columns. Each row corresponds to one email message. The first column is the _response_ variable and describes whether a message is spam `1` or ham `0`. The remaining 54 columns are _features_ that you will use to build a classifier. These features correspond to 54 different keywords (such as "money", "free", and "receive") and special characters (such as ":", "!", and "$"). A feature has the value `1` if the keyword appears in the message and `0` otherwise.

As mentioned there is also a 500 row set of *test data*. It contains the same 55 columns.

In [2]:
testing_spam = np.loadtxt(open("data/testing_spam.csv"), delimiter=",").astype(int)
print("Shape of the spam testing data set:", testing_spam.shape)
print(testing_spam)

Shape of the spam testing data set: (500, 55)
[[1 0 0 ... 1 1 1]
 [1 1 0 ... 1 1 1]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 1 0 0]
 [0 0 0 ... 1 0 0]]


## Part One
Write all of the code for your classifier below this cell. There is some very rough skeleton code in the cell directly below. You may insert more cells below this if you wish, but you must not duplicate any cells as this can break the grading script.

### Submission Requirements
Your code must provide a variable with the name `classifier`. This object must have a method called `predict` which takes input data and returns class predictions. The input will be a single $n \times 54$ numpy array, your classifier should return a numpy array of length $n$ with classifications. There is a demo in the cell below, and a test you can run before submitting to check your code is working correctly.

Your code must run on our test machine in under 30 seconds. If you wish to train a more complicated model (e.g. neural network) which will take longer, you are welcome to save the model's weights as a file and then load these in the cell below so we can test it. You must include the code which computes the original weights, but this must not run when we run the notebook – comment out the code which actually executes the routine and make sure it is clear what we need to change to get it to run. Remember that we will be testing your final classifier on additional hidden data.

In [3]:
# Code to demonstrate how I optimised all my hyperparameters locaed in hyperparam_optimiser.py


class NaiveBayesClassifier:
    def __init__(self, training_data, testing_data):
        self.training_data = training_data.astype(int)
        self.testing_data = testing_data
        self.prior_normal_probs = {}
    
    
    def probability_calc(self):
        label_arr = self.training_data[:, 0]
        # Initialise dictionaries for storing prior probabilities and feature probabilities
        prior_probs = {}
        all_feature_probs = {}
        # Laplace smoothing factor to avoid division by zero
        laplace_alpha = 1
        unique_labels = np.unique(label_arr)
        
        for label in unique_labels:
            # Count occurrences of each class label to calculate prior probabilities
            label_count = np.sum(label == label_arr)
            prior_probs[label] = np.log(label_count / len(label_arr))
            self.prior_normal_probs[label] = label_count / len(label_arr)
            probabilities = np.array([])
    
            # Initialise dictionary for storing probabilities of each feature given the class
            feature_probs = {}
            # Extract rows corresponding to the current class
            features = self.training_data[self.training_data[:, 0] == label][:, 1:]
    
            for feature in range(np.shape(features[0])[0]):  # Loop through all features 
                # Calculate probability of each feature given the class with Laplace smoothing
                prob = np.log((np.sum(features[:, feature]) + laplace_alpha) / \
                       (len(features) + laplace_alpha * 2))  # Corrected smoothing formula
                probabilities = np.append(probabilities, prob) 
            
            all_feature_probs[label] = probabilities  
            
        return prior_probs, all_feature_probs



    def testing(self, prior_probs, all_feature_probs, test_element):
        label_probs = {}
        for label in np.unique(self.training_data[:, 0]):
            probabilities = np.array([])

            # Skip label in test_element and consider feature if present - calculate probabilities 
            for feature in range(len(test_element)): 
                if test_element[feature] == 1: 
                    feature_prob = all_feature_probs[label][feature]  
                    final_prob = prior_probs[label] + feature_prob
                    probabilities = np.append(probabilities, final_prob)
            
            # Calculate class probability by multiplying prior with product of feature probabilities
            label_probs[label] = np.sum(probabilities)
    
        return label_probs

    

    def probability_normalisation(self, log_probs):
        # Converts log probabilities to normalised probabilities 
        e_x = np.exp(log_probs - np.max(log_probs))
        return e_x / e_x.sum(axis=0)  

    
    def predictor(self):
        prior_probs, all_feature_probs = self.probability_calc()
        normalised_probs = []

        for test_element in self.testing_data:
            log_probs = self.testing(prior_probs, all_feature_probs, test_element)
            normalised_prob = self.probability_normalisation(np.array(list(log_probs.values())))
            normalised_probs.append(normalised_prob)

        normalised_probs = np.array(normalised_probs)
        
        # Find the indices of the max values in each row
        max_indices = np.argmax(normalised_probs, axis=1)
        
        # Initialise an array to hold the transformed values
        transformed_values = np.zeros_like(max_indices, dtype=float)
        
        # Iterate over each row
        for i, index in enumerate(max_indices):
            if index == 0:  # If the max value comes from the second column
                transformed_values[i] = 1 - normalised_probs[i, index]
            else:  # If the max value comes from the first column
                transformed_values[i] = normalised_probs[i, index]

        return transformed_values, self.prior_normal_probs






class LogisticRegressionClassifier:
    def __init__(self, training_data, training_cycles, learning_rate, lr_bias, buffer_divisor, lambda_penalty):
        self.training_data = training_data
        self.training_cycles = training_cycles
        self.learning_rate = learning_rate
        self.lr_bias = lr_bias
        self.buffer_divisor = buffer_divisor
        self.lambda_penalty = lambda_penalty
    

    def train(self, features, constant, training_features, training_labels):
        errors = [] 
        train_count = training_features.shape[1] 

        for cycle in range(self.training_cycles):
            
            # Calculate the linear combination of the features multiplied by their respected weights plus the constant. Feature array transpose needed for multiplication 
            logistic_input = np.dot(features.T, training_features) + constant
            probability = self.sigmoid_function(logistic_input)
            
            # Feature gradient decent using Lasso regularisation to add a penalty equal to the absolute value of the magnitude of feature values. 
            feature_gradient = (1 / train_count) * np.dot(training_features, (probability - training_labels).T)
            feature_gradient += (self.lambda_penalty / train_count) * np.sign(features)

            # Constant gradient decent 
            constant_gradient = (1 / train_count) * np.sum(probability - training_labels)

            # Update the values of the features and constant weightings
            features = features - self.learning_rate * feature_gradient
            constant = constant - self.learning_rate * constant_gradient

        return features, constant


    
    def sigmoid_function(self, logistic_input):
        # Predict the probability that logistic_input belongs to class 1. Clip the input to prevent overflow of small numbers.
        logistic_input = np.clip(logistic_input, -1000, 1000) 
        probabilities = 1 / (1 + np.exp(-logistic_input))
        
        return probabilities


    
    def classify(self, features, constant, test_data):
        # Create array for prediction same length as the amount of test_data elements
        test_data_count = test_data.shape[1]  

        # Compute logistic predictions using trained features weight and trained constant 
        probabilities = self.sigmoid_function(np.dot(features.T, test_data) + constant)

        return probabilities
    

    
    def predict(self, test_data):

        # Split the training data 
        training_labels = self.training_data[:, 0].astype(int)
        training_features = self.training_data[:, 1:].astype(int)
        
        # Change the shape of the feature and training arrays so they can be used for matrix math calculations 
        training_features = training_features.T
        training_labels = training_labels.reshape(1, -1)
        
        # Train the model using gradient descent
        default_features = np.zeros((training_features.shape[0], 1))
        default_constant = 0
        features, constant = self.train(default_features, default_constant, training_features, training_labels)
        
        # Make predictions on the test data
        lr_probabilities = self.classify(features, constant, test_data.T)
        
        # Instantiate the bayes classifier, run predict and return probabilities 
        naive_bayes = NaiveBayesClassifier(self.training_data, test_data)
        bayes_probabilities, prior_probabilities = naive_bayes.predictor()

        # Combining the probabilities 
        same_side_condition = ((lr_probabilities > 0.5) & (bayes_probabilities > 0.5)) | ((lr_probabilities < 0.5) & (bayes_probabilities < 0.5))
        average_values = (lr_probabilities * self.lr_bias + bayes_probabilities) / 2
        combined_probabilities = np.where(same_side_condition, average_values, lr_probabilities)
        
        # Using prior probs to add a bias within the buffer to 0.5
        max_key = max(prior_probabilities, key=prior_probabilities.get)
        max_value = prior_probabilities[max_key]
        buffer = max_value - 0.5
        
        # Adjusting decision thresholds based on the buffer
        upper_threshold = 0.5 + buffer / self.buffer_divisor
        lower_threshold = 0.5 - buffer / self.buffer_divisor
        
        # Condition to check if probabilities are decisively above or below the adjusted thresholds
        buffer_condition = (combined_probabilities > upper_threshold) | (combined_probabilities < lower_threshold)
        
        # Keep lr_probabilities where buffer_condition is true, otherwise, use a default probability
        combined_probabilities_buf = np.where(buffer_condition, combined_probabilities, max_value)
        
        # Generating predictions based on the adjusted combined_probabilities
        predictions = (combined_probabilities_buf > 0.5).astype(int)

        return predictions



def create_classifier():
    classifier = LogisticRegressionClassifier(training_spam, 9000, 0.32, 0.5, 0.58, -0.38)
    return classifier

classifier = create_classifier()

### Accuracy Estimate
In the cell below there is a function called `my_accuracy_estimate()` which returns `0.5`. Before you submit the assignment, write your best guess for the accuracy of your classifier into this function, as a percentage between `0` and `1`. So if you think you will get 80% of inputs correct, return the value `0.8`. This will form a small part of the marking criteria for the assignment, to encourage you to test your own code.

In [4]:
def my_accuracy_estimate():
    return 0.92

Write all of the code for your classifier above this cell.

### Testing Details
Your classifier will be tested against some hidden data from the same source as the original. The accuracy (percentage of classifications correct) will be calculated, then benchmarked against common methods. At the very high end of the grading scale, your accuracy will also be compared to the best submissions from other students (in your own cohort and others!). Your estimate from the cell above will also factor in, and you will be rewarded for being close to your actual accuracy (overestimates and underestimates will be treated the same).

#### Test Cell
The following code will run your classifier against the provided test data. To enable it, set the constant `SKIP_TESTS` to `False`.

The original skeleton code above classifies every row as ham, but once you have written your own classifier you can run this cell again to test it. So long as your code sets up a variable called `classifier` with a method called `predict`, the test code will be able to run. 

Of course you may wish to test your classifier in additional ways, but you *must* ensure this version still runs before submitting.

**IMPORTANT**: you must set `SKIP_TESTS` back to `True` before submitting this file!

In [5]:
SKIP_TESTS = True

if not SKIP_TESTS:
    testing_spam = np.loadtxt(open("data/testing_spam.csv"), delimiter=",").astype(int)
    test_data = testing_spam[:, 1:]
    test_labels = testing_spam[:, 0]

    predictions = classifier.predict(test_data)
    accuracy = np.count_nonzero(predictions == test_labels)/test_labels.shape[0]
    print(f"Accuracy on test data is: {accuracy}")

In [6]:
import sys
import pathlib

fail = False;

success = '\033[1;32m[✓]\033[0m'
issue = '\033[1;33m[!]'
error = '\033[1;31m\t✗'

#######
##
## Skip Tests check.
##
## Test to ensure the SKIP_TESTS variable is set to True to prevent it slowing down the automarker.
##
#######

if not SKIP_TESTS:
    fail = True;
    print("{} \'SKIP_TESTS\' is incorrectly set to False.\033[0m".format(issue))
    print("{} You must set the SKIP_TESTS constant to True in the cell above.\033[0m".format(error))
else:
    print('{} \'SKIP_TESTS\' is set to true.\033[0m'.format(success))

#######
##
## File Name check.
##
## Test to ensure file has the correct name. This is important for the marking system to correctly process the submission.
##
#######
    
p3 = pathlib.Path('./spamclassifier.ipynb')
if not p3.is_file():
    fail = True
    print("{} The notebook name is incorrect.\033[0m".format(issue))
    print("{} This notebook file must be named spamclassifier.ipynb\033[0m".format(error))
else:
    print('{} The notebook name is correct.\033[0m'.format(success))

#######
##
## Create classifier function check.
##
## Test that checks the create_classifier function exists. The function should train the classifier and return it so that it can be evaluated by the marking system.
##
#######

if "create_classifier" not in dir():
    fail = True;
    print("{} The create_classifier function has not been defined.\033[0m".format(issue))
    print("{} Your code must include a create_classifier function as described in the coursework specification.\033[0m".format(error))
    print("{} If you believe you have, \'restart & run-all\' to clear this error.\033[0m".format(error))
else:
    print('{} The create_classifier function has been defined.\033[0m'.format(success))

#######
##
## Classifier variable check.
##
## Test that checks the classifier variable exists. The marking system will use this variable to make predictions based on a set of random features you have not seen. Your score will be based on how well your classifier predicts the hidden labels.
##
#######

if 'classifier' not in vars():
    fail = True;
    print("{} The classifer variable has not been defined.\033[0m".format(issue))
    print("{} Your code must create a variable called \'classifier\' as described in the coursework specification.\033[0m".format(error))
    print("{} This variable should contain the trained classifier you have created.\033[0m".format(error))
else:
    print('{} The classifer variable has been correctly defined.\033[0m'.format(success))

#######
##
## Accuracy Estimation check.
##
## Test that checks the accuracy estimation function exists and is a reasonable value. This is a requirement of the coursework specification and is used by the marking system when generating your final grade.
##
#######

if "my_accuracy_estimate" not in dir():
    fail = True;
    print("{} The my_accuracy_estimate function has not been defined.\033[0m".format(issue))
    print("{} Your code must include a my_accuracy_estimate function as described in the coursework specification.\033[0m".format(error))
    print("{} If you believe you have, \'restart & run-all\' to clear this error.\033[0m".format(error))
else:
    if my_accuracy_estimate() == 0.5:
        print("{} my_accuracy_estimate function warning.\033[0m".format(issue))
        print("{} my_accuracy_estimate function returns a value of 0.5 - Your classifier is no better than random chance.\033[0m".format(error))
        print("{} Are you sure this is correct.\033[0m".format(error))
    else:
        print('{} The my_accuracy_estimate function has been defined correctly.\033[0m'.format(success))

#######
##
## Test set check.
##
## Test that checks your classifier actually works. The calls made here are the same made by the automarker - albeit with different data. If your work fails this test it will score 0 in the automarker.
##
#######

try:
    testing_spam = np.loadtxt(open("data/testing_spam.csv"), delimiter=",").astype(int)
    test_data = testing_spam[:, 1:]
    test_labels = testing_spam[:, 0]
    
    try:
        predictions = classifier.predict(test_data)
        accuracy = np.count_nonzero(predictions == test_labels)/test_labels.shape[0]
        print('{0} Success running test set - Accuracy was {1:.2f}%.\033[0m'.format(success, (accuracy*100)))
    except Exception as e:
        fail = True
        print("{} Error running test set.\033[0m".format(issue))
        print("{} Your code produced the following error. This error will result in a zero from the automarker, please fix.\033[0m".format(error))
#         print("{} {}\033[0m".format(error, e))
        print(e)
except:
    sys.stderr.write("Unable to run one test as the file \'data/testing_spam.csv\' could not be found.")

#######
##
## Final Summary
##
## Prints the final results of the submission tests.
##
#######

if fail:
    sys.stderr.write("Your submission is not ready! Please read and follow the instructions above.")
else:
    print("\033[1m\n\n")
    print("╔═══════════════════════════════════════════════════════════════╗")
    print("║                        Congratulations!                       ║")
    print("║                                                               ║")
    print("║            Your work meets all the required criteria          ║")
    print("║                   and is ready for submission.                ║")
    print("╚═══════════════════════════════════════════════════════════════╝")
    print("\033[0m")
    

[✓] 'SKIP_TESTS' is set to true.
[✓] The notebook name is correct.
[✓] The create_classifier function has been defined.
[✓] The classifer variable has been correctly defined.
[✓] The my_accuracy_estimate function has been defined correctly.
[✓] Success running test set - Accuracy was 95.00%.



╔═══════════════════════════════════════════════════════════════╗
║                        Congratulations!                       ║
║                                                               ║
║            Your work meets all the required criteria          ║
║                   and is ready for submission.                ║
╚═══════════════════════════════════════════════════════════════╝



In [7]:
# This is a test cell. Please do not modify or delete.